In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName("myTrain") \
.master("local[4]") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

# Veri Setini okuma

In [4]:
df = spark.read.format("csv") \
.option("header",True) \
.option("inferScHEMA",True) \
.option("sep",",") \
.load("/home/taha/Downloads/Advertising.csv")

In [7]:
df.toPandas().head()

,_c0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


# Stun Isımlerini  Degistirme

In [8]:
yeni_nitelikler = ["id","TV","Radio","Newspaper","label"]

In [16]:
df2 = df.selectExpr("_c0 as id","TV","Radio","Newspaper","Sales as label")
df2.toPandas().head()

,id,TV,Radio,Newspaper,label
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


# Nitelikleri Siniflandirma

In [39]:
numerik_nitelikler = ["TV","Radio"]
# newspapaer cikariliyor
label=["label"]

In [40]:
df2.describe().toPandas().head()

,summary,id,TV,Radio,Newspaper,label
0,count,200,200,200,200,200
1,mean,100.5,147.0425,23.264000000000024,30.553999999999995,14.022500000000003
2,stddev,57.879184513951124,85.85423631490805,14.846809176168728,21.77862083852283,5.217456565710477
3,min,1,0.7,0.0,0.3,1.6
4,max,200,296.4,49.6,114.0,27.0


# Eksik bir veri yok Veri kaybı Gozukmuyor

# Veri Hazirligine Gecebilirz

In [41]:
from pyspark.ml.feature import VectorAssembler

In [42]:
vector_assembler = VectorAssembler().setInputCols(numerik_nitelikler) \
.setOutputCol("features")

# Regresyon Modeli

In [43]:
from pyspark.ml.regression import LinearRegression

In [44]:
lr_obj = LinearRegression().setFeaturesCol("features") \
.setLabelCol("label")

# Pipeline Modeli

In [45]:
from pyspark.ml import Pipeline

In [46]:
pipeline_obj = Pipeline().setStages([vector_assembler,lr_obj])

# Veri Ayirimi

In [47]:
train_df , test_df = df2.randomSplit([0.8,0.2],seed = 142)

# Model Egitimi

In [48]:
pipeline_model = pipeline_obj.fit(train_df)

# Model testi

In [49]:
result_df = pipeline_model.transform(test_df)

In [50]:
result_df.toPandas().head()

,id,TV,Radio,Newspaper,label,features,prediction
0,3,17.2,45.9,69.3,9.3,"[17.2, 45.9]",12.773880
1,6,8.7,48.9,75.0,7.2,"[8.7, 48.9]",12.991416
2,9,8.6,2.1,1.0,4.8,"[8.6, 2.1]",3.731130
3,10,199.8,2.6,21.2,10.6,"[199.8, 2.6]",12.283048
4,17,67.8,36.6,114.0,12.5,"[67.8, 36.6]",13.171620


# Pipeline modelin icinden lr modeli almak

In [51]:
lr_model = pipeline_model.stages[1]

In [52]:
lr_model.coefficients

DenseVector([0.0442, 0.1978])

In [53]:
lr_model.intercept

2.935593134859488

In [54]:
lr_model.summary.r2

0.8928931248714045

In [55]:
lr_model.summary.rootMeanSquaredError

1.6578475603790448

In [56]:
lr_model.summary.pValues

[0.0, 0.0, 3.774758283725532e-15]

In [57]:
lr_model.summary.tValues

[27.918094216203865, 21.216582516976807, 8.740412243937218]

# NOT
### p values lere baktigimizda newspaperin degeiriniz(0.57)
### yani 0.05 ten kat kat buyuk oldugu gorulur
### daha kesin sonuc icin bu deger cikarilmalidir
### simdi numerik degisikliklerden bu degeri cikaracagiz
### ve kodu tekrar calistaracagiz

### Cikar isleminden sonra p degerleri 0.05 ten az kaldi

# Prediction

In [58]:
# 100 bin TV için 10 bin Radio için

In [59]:
import pandas

In [61]:
new_df = {"TV":[100.0],"Radio":[10.0]}
pd_df = pandas.DataFrame(data=new_df)
pd_df.head()

,TV,Radio
0,100.0,10.0


# pandas dataframe i spark dataframe cevirmeliyiz

In [64]:
predict_df = spark.createDataFrame(pd_df)

In [65]:
predict_df.show()

+-----+-----+
|   TV|Radio|
+-----+-----+
|100.0| 10.0|
+-----+-----+



In [66]:
predict_df_vec = vector_assembler.transform(predict_df)

In [68]:
lr_model.transform(predict_df_vec).show()

+-----+-----+------------+-----------------+
|   TV|Radio|    features|       prediction|
+-----+-----+------------+-----------------+
|100.0| 10.0|[100.0,10.0]|9.334383277881834|
+-----+-----+------------+-----------------+

